# WebScraping of tribunal transcript cases

This notebook aims to show the functionality for cleaning transcripts of cases through web scraping.

Following the general ethical guidelines when using WebScraping, it was retrieved the permission on the three tribunals:
- [International criminal tribunal for the former Yugoslavia](https://www.icty.org/), permission can be fetched [here](https://www.icty.org/robots.txt)
- [Extraordinary Chamber in the Courts of Cambodia](https://www.eccc.gov.kh/), permission can be fetched [here](https://www.eccc.gov.kh/robots.txt)
- [International Criminal Tribunal for Rwanda](https://ucr.irmct.org/) no robots.txt file was found

### Imports

In [ ]:
#import sys  
#!{sys.executable} -m pip install PyPDF2
#!{sys.executable} -m pip install openpyxl
#!{sys.executable} -m pip install python-docx # didnt work with doc files
#!{sys.executable} -m pip install textract

Python 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 

In [1]:
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup
from os import listdir
from os.path import isfile, join
from PyPDF2 import PdfFileReader
import pandas as pd
import docx
import textract

import src.cleaning_transcripts as cleaning_transc

### Globals

In [2]:
GLB_FILE_ENCODING_UTF8 = "utf8"
GLB_FILE_WRITE_OP = "w"
GLB_FILE_BINARY_OP = "wb"

GLB_HTML_P_TAG = "p"
GLB_HTML_PARSER = "html.parser"

GLB_PATH_OUTPUT_DIRECTORY_ICTY = "output/clean_transcripts/icty"
GLB_PATH_INPUT_DIRECTORY_ECCC = "input/transcripts/eccc"
GLB_PATH_OUTPUT_DIRECTORY_ECCC = "output/clean_transcripts/eccc"
GLB_PATH_INPUT_DIRECTORY_ICTR = "input/transcripts/ictr"
GLB_PATH_OUTPUT_DIRECTORY_ICTR = "output/clean_transcripts/ictr"
GLB_EXTENSION_TXT = ".txt"

GLB_CHAR_NEWLINE = "\n"

GLB_COURT_PREFIX_FILE_ICTY = "ICTY_"
GLB_COURT_PREFIX_FILE_ECCC = "ECCC_"
GLB_COURT_PREFIX_FILE_ICTR = "ICTR_"

DEBUG = True
GLB_GET_LIST_FILES_ECCC_FROM_EXCEL_FILE = True
GLB_GET_LIST_FILES_ICTR_FROM_EXCEL_FILE = True
GLB_GET_LIST_FILES_ICTY_FROM_EXCEL_FILE = True

## International criminal tribunal for the former Yugoslavia

In [78]:
#transcript_icty_case_url = "https://www.icty.org/x/cases/tadic/trans/en/960719ed.htm"
#transcript_icty_case_url = "https://www.icty.org/x/cases/tolimir/trans/en/100322IT.htm"
#transcript_icty_case_url = "https://www.icty.org/x/cases/mladic/trans/en/120903IT.htm"
#transcript_icty_case_url = "https://www.icty.org/x/cases/mladic/trans/en/120906ED.htm"
transcript_icty_case_url = "https://www.icty.org/x/cases/milan_lukic_sredoje_lukic/trans/en/080919ED.htm"

In [98]:
if DEBUG:
    # Extracting information from the "International criminal tribunal for the former Yugoslavia"
    response = requests.get(transcript_icty_case_url)
    
    #print(f"{response.text}")
    
    # Get just the content of the page
    #response.__dict__["_content"]

    content_soup = BeautifulSoup(response.content, "html.parser")

    # Print in text content in html format
    #print(content_soup.prettify())

    # Get list of paragraphs (this is how the information is retrieved)
    list_p = list()
    for p in content_soup.find_all("p"):
        list_p.append(p)
    print(f'Number of retrieved paragraphs of transcript {transcript_icty_case_url} is {len(list_p)}')
    print("="*50)

    counter = 0
    for paragraph in list_p:
        clean_paragraph = cleaning_transc.cleanParagraphsICFYtranscript(str(paragraph))
        if clean_paragraph != cleaning_transc.GLB_EMPTY_STRING:
            counter+=1
            print(f"{clean_paragraph}")

Number of retrieved paragraphs of transcript https://www.icty.org/x/cases/milan_lukic_sredoje_lukic/trans/en/080919ED.htm is 1717
Friday,  19 September 2008
[Closed session]
<span lang="FR" style='font-size:10.0pt;line-height:200%;font-family:"Courier New"'>11  Pages 1914-1948 redacted. Closed session.
[Open session]
THE REGISTRAR:  Your Honours, we're in open session.
JUDGE ROBINSON:  Ms. Sartorio, we will be able to conclude this
witness's testimony today?  That's a question.
MS. SARTORIO:  I'm -- I'm confident -- hopeful that we can, and I
think we can, Your Honour.
JUDGE ROBINSON:  Yes.  Thank you.
[The witness entered court]
WITNESS:  NURKO DERVISEVIC
[Witness answered through interpreter]
JUDGE ROBINSON:  Madam Usher, you may read the declaration to the
witness.
THE WITNESS: [Interpretation] I solemnly declare that I will
speak the truth, the whole truth, and nothing but the truth.
JUDGE ROBINSON:  You may sit.  You may have a seat.  You may
begin, Ms. Sartorio.
MS. SARTORIO:  I'

#### Get list of documents (either from excel file or default values)

In [99]:
if GLB_GET_LIST_FILES_ICTY_FROM_EXCEL_FILE:
    PATH_ICTY_EXCEL_LIST_FILE_ICTY_TRANSCRIPTS = "input/html-links-ICTY.xlsx"

    df = pd.read_excel(PATH_ICTY_EXCEL_LIST_FILE_ICTY_TRANSCRIPTS, index_col=None, header=None, names=["url", "court", "accused", "case", "date", "witness"])

    list_url_cases_icty = list(df["url"])
else:
    list_url_cases_icty = [transcript_eccc_case_url]
    
print(f"Length of elements in list_url_cases_icty is {len(list_url_cases_icty)}")

Length of elements in list_url_cases_icty is 50


#### Save Documents

In [101]:
for index_case, url_html_case in enumerate(list_url_cases_icty):
    response = requests.get(url_html_case)
    
    content_soup = BeautifulSoup(response.content, GLB_HTML_PARSER)
    
    list_p = list()
    for p in content_soup.find_all(GLB_HTML_P_TAG):
        list_p.append(p)
        
    counter = 0
    id_case = url_html_case[url_html_case.rindex("/")+1: url_html_case.rindex(".")]
    f = open(join(GLB_PATH_OUTPUT_DIRECTORY_ICTY, GLB_COURT_PREFIX_FILE_ICTY + id_case + GLB_EXTENSION_TXT), GLB_FILE_WRITE_OP, encoding=GLB_FILE_ENCODING_UTF8)
    for paragraph in list_p:
        clean_paragraph = cleaning_transc.cleanParagraphsICFYtranscript(str(paragraph))
        if clean_paragraph != cleaning_transc.GLB_EMPTY_STRING:
            counter+=1
            f.write(clean_paragraph + GLB_CHAR_NEWLINE)
    f.close()
        
    print(f'{index_case+1}) Number of retrieved paragraphs of transcript {transcript_icty_case_url} is {len(list_p)} was reduced to {counter}')
    

1) Number of retrieved paragraphs of transcript https://www.icty.org/x/cases/milan_lukic_sredoje_lukic/trans/en/080919ED.htm is 2315 was reduced to 2194
2) Number of retrieved paragraphs of transcript https://www.icty.org/x/cases/milan_lukic_sredoje_lukic/trans/en/080919ED.htm is 2185 was reduced to 2078
3) Number of retrieved paragraphs of transcript https://www.icty.org/x/cases/milan_lukic_sredoje_lukic/trans/en/080919ED.htm is 2185 was reduced to 2078
4) Number of retrieved paragraphs of transcript https://www.icty.org/x/cases/milan_lukic_sredoje_lukic/trans/en/080919ED.htm is 2264 was reduced to 1917
5) Number of retrieved paragraphs of transcript https://www.icty.org/x/cases/milan_lukic_sredoje_lukic/trans/en/080919ED.htm is 2264 was reduced to 1917
6) Number of retrieved paragraphs of transcript https://www.icty.org/x/cases/milan_lukic_sredoje_lukic/trans/en/080919ED.htm is 2264 was reduced to 1974
7) Number of retrieved paragraphs of transcript https://www.icty.org/x/cases/milan

In [28]:
print(f'Duplicated values: {df[df["url"].duplicated()]["url"].count()}')
df[df["url"].duplicated()]["url"]

Duplicated values: 17


2     https://www.icty.org/x/cases/milan_lukic_sredo...
4     https://www.icty.org/x/cases/milan_lukic_sredo...
6     https://www.icty.org/x/cases/milan_lukic_sredo...
12    https://www.icty.org/x/cases/mladic/trans/en/1...
26    https://www.icty.org/x/cases/popovic/trans/en/...
34    https://www.icty.org/x/cases/tolimir/trans/en/...
38    https://www.icty.org/x/cases/tolimir/trans/en/...
40    https://www.icty.org/x/cases/popovic/trans/en/...
41    https://www.icty.org/x/cases/popovic/trans/en/...
42    https://www.icty.org/x/cases/popovic/trans/en/...
43    https://www.icty.org/x/cases/popovic/trans/en/...
44    https://www.icty.org/x/cases/popovic/trans/en/...
45    https://www.icty.org/x/cases/popovic/trans/en/...
46    https://www.icty.org/x/cases/popovic/trans/en/...
47    https://www.icty.org/x/cases/popovic/trans/en/...
48    https://www.icty.org/x/cases/popovic/trans/en/...
49    https://www.icty.org/x/cases/popovic/trans/en/...
Name: url, dtype: object

## Extraordinary Chamber in the Courts of Cambodia

In [12]:
transcript_eccc_case_url = "https://www.eccc.gov.kh/sites/default/files/documents/courtdoc/%5Bdate-in-tz%5D/E1_41.1_TR001_20090701_Final_EN_Pub.pdf"

In [25]:
str_tmp = ''
pattern_was_found = False

if DEBUG:
    counter = 0
    list_all_sentences = list()
    # Extracting information from the "Extraordinary Chamber in the Courts of Cambodia"
    # Get the PDF
    response = requests.get(transcript_eccc_case_url)
    doc_name = transcript_eccc_case_url[transcript_eccc_case_url.rindex("/")+1:]
    
    f = open(join(GLB_PATH_INPUT_DIRECTORY_ECCC, doc_name), GLB_FILE_BINARY_OP)
    f.write(response.content)
    f.close()    
    
    # Get content of the PDF
    pdf = PdfFileReader(join(GLB_PATH_INPUT_DIRECTORY_ECCC, doc_name))
    number_of_pages = len(pdf.pages)
    #print(f"Number of pages {number_of_pages}")
    index_page_of_interest = 1
    for PATTERN_BEGIN in cleaning_transc.GLB_ECCC_PATTERN_BEGIN_CONTENT_LIST:
        if pattern_was_found:
            break
            
        for index_page in range(number_of_pages):
            page_pdf = pdf.pages[index_page]
            text_page = page_pdf.extract_text()

            if not pattern_was_found:
                index_pattern = text_page.index(PATTERN_BEGIN) if PATTERN_BEGIN in text_page else -1
                
                if index_pattern >= 0:
                    pattern_was_found = True

                    list_aux = cleaning_transc.cleanPagePdfECCCtranscript(text_page, index_page_of_interest)
                    list_all_sentences = [*list_all_sentences, *list_aux]
                    index_page_of_interest += 1
            else:
                list_aux = cleaning_transc.cleanPagePdfECCCtranscript(text_page, index_page_of_interest)
                list_all_sentences = [*list_all_sentences, *list_aux]
                index_page_of_interest += 1
                
    if not pattern_was_found:
        print(f"Pattern was never found for document: {transcript_eccc_case_url}")
    else:
        print(f"Total num of sentences from PDF file {len(list_all_sentences)}")
        print("="*50)

        for e in list_all_sentences:
            #print("====================BEGIN====================")
            #print(f'<<{e}>>')
            clean_sent = cleaning_transc.cleanSentenceECCCtranscript(e)
            if clean_sent != cleaning_transc.GLB_EMPTY_STRING :
                counter+=1
                print(clean_sent)
            #print("=====================END=====================")

        print(f"Num new sentences <<{counter}>> reduced from {len(list_all_sentences)}")

Total num of sentences from PDF file 2600
P R O C E E D I N G S
(Judges enter courtroom)
MR. PRESIDENT:
Please be seated.  The Court is now in session.
According to our schedule, today we're going to hear the
testimony of another survivor; the third person among the nine
survivors of S-21.
The lawyer, I note your presence.  Would you like to make any
comments?
MS. STUDZINSKY:
Mr. President, good morning.  Your Honours, good morning, dear
colleagues.
Yes, I would like to make some observations and also I'm seeking
for clarification before we hear the next survivor.
We have observed that Mr. Chum Mey yesterday was overwhelmed
sometimes when he accounted his story and he had to cry, and he
could not control his emotions any more.  He shares his
traumatization as well as the next survivor, who is my client
together with Cambodian colleagues, and he shares this situation
with other survivors, victims, civil parties and witnesses.
I would like to make a proposal.  I would like that the Chamb

#### Get list of documents (either from excel file or default values)

In [75]:
if GLB_GET_LIST_FILES_ECCC_FROM_EXCEL_FILE:
    PATH_ECCC_EXCEL_LIST_FILE_ECCC_TRANSCRIPTS = "input/html-links-ECCC.xlsx"

    df = pd.read_excel(PATH_ECCC_EXCEL_LIST_FILE_ECCC_TRANSCRIPTS, index_col=None, header=None, names=["url", "court", "case", "id_transcript", "date", "person_name"])

    list_url_cases_eccc = list(df["url"])
else:
    list_url_cases_eccc = [transcript_eccc_case_url]
    
print(f"Length of elements in list_url_cases_eccc is {len(list_url_cases_eccc)}")

FileNotFoundError: [Errno 2] No such file or directory: 'input/html-links-ECCC.xlsx'

#### Save Documents

In [ ]:
counter = 0
pattern_was_found = False

for index_case, url_html_case in enumerate(list_url_cases_eccc):
    list_all_sentences = list()
    
    response = requests.get(url_html_case)
    doc_name = url_html_case[url_html_case.rindex("/")+1:]
    if doc_name in ["E1_300.1_TR002_20150518_Final_EN_Pub.pdf"]: #Exceptions
        continue
    
    # Write PDF
    f = open(join(GLB_PATH_INPUT_DIRECTORY_ECCC, doc_name), GLB_FILE_BINARY_OP)
    f.write(response.content)
    f.close()
    
    # Get content of the PDF
    pdf = PdfFileReader(join(GLB_PATH_INPUT_DIRECTORY_ECCC, doc_name))
    number_of_pages = len(pdf.pages)
    
    index_page_of_interest = 1
    pattern_was_found = False
    
    for PATTERN_BEGIN in cleaning_transc.GLB_ECCC_PATTERN_BEGIN_CONTENT_LIST:
        if pattern_was_found:
            break
            
        for index_page in range(number_of_pages):
            page_pdf = pdf.pages[index_page]
            text_page = page_pdf.extract_text()

            if not pattern_was_found:
                index_pattern = text_page.index(PATTERN_BEGIN) if PATTERN_BEGIN in text_page else -1

                if index_pattern >= 0:
                    pattern_was_found = True

                    list_aux = cleaning_transc.cleanPagePdfECCCtranscript(text_page, index_page_of_interest)
                    
                    if list_aux == None:
                        pattern_was_found = False
                        continue
                        
                    list_all_sentences = [*list_all_sentences, *list_aux]
                    index_page_of_interest += 1
            else:
                list_aux = cleaning_transc.cleanPagePdfECCCtranscript(text_page, index_page_of_interest)
                list_all_sentences = [*list_all_sentences, *list_aux]
                index_page_of_interest += 1
    
    if not pattern_was_found:
        print(f"Pattern was never found for document: {url_html_case}")
        continue
        
    counter = 0
    id_case = url_html_case[url_html_case.rindex("/")+1: url_html_case.rindex(".")]
    f = open(join(GLB_PATH_OUTPUT_DIRECTORY_ECCC, GLB_COURT_PREFIX_FILE_ECCC + id_case + GLB_EXTENSION_TXT), GLB_FILE_WRITE_OP, encoding=GLB_FILE_ENCODING_UTF8)
    
    for index, sentence in enumerate(list_all_sentences):
        clean_paragraph = cleaning_transc.cleanSentenceECCCtranscript(sentence)
        if clean_paragraph != cleaning_transc.GLB_EMPTY_STRING:
            counter+=1
            f.write(clean_paragraph + GLB_CHAR_NEWLINE)
    f.close()
            
    print(f'{index_case+1}) Number of retrieved paragraphs of transcript {url_html_case} is {len(list_all_sentences)} was reduced to {counter}')
    

## International Criminal Tribunal for Rwanda

In [4]:
#transcript_ictr_case_url = "https://ucr.irmct.org/LegalRef/CMSDocStore/Public/English/Transcript/NotIndexable/ICTR-96-04/TRS13317R0000613662.PDF"
transcript_ictr_case_url = "https://ucr.irmct.org/LegalRef/CMSDocStore/Public/English/Transcript/NotIndexable/ICTR-98-44/TRS15978R0000616228.DOC"
#list_url_cases_ictr = [transcript_ictr_case_url]

In [46]:
if DEBUG:
    list_all_sentences = list()
    # Extracting information from the "International Criminal Tribunal for Rwanda"
    # Get the PDF
    response = requests.get(transcript_ictr_case_url)
    doc_name = transcript_ictr_case_url[transcript_ictr_case_url.rindex("/")+1:]
    
    f = open(join(GLB_PATH_INPUT_DIRECTORY_ICTR, doc_name), GLB_FILE_BINARY_OP)
    f.write(response.content)
    f.close()
    
    # Get content of the DOC
    """
    pdf = PdfFileReader(join(GLB_PATH_INPUT_DIRECTORY_ICTR, doc_name))
    number_of_pages = len(pdf.pages)
    """
    print(f"Path {join(GLB_PATH_INPUT_DIRECTORY_ICTR, doc_name)}")
    text = textract.process(join(GLB_PATH_INPUT_DIRECTORY_ICTR, doc_name))
    text = text.decode("utf-8")
    print("Num of sentences", len(text.split('\n')))
    # Consider from PROCEEDINGS
    patternWasFound = False
    for PATTERN_PROCEEDINGS in cleaning_transc.GLB_ECCC_PATTERN_BEGIN_CONTENT_LIST:
        if not patternWasFound:
            index_pattern = text.index(PATTERN_PROCEEDINGS) if PATTERN_PROCEEDINGS in text else -1
            if index_pattern >= 0:
                patternWasFound = True
                text = text[index_pattern:]
                break
    
    text = text.split("\n")
    text = [cleaning_transc.cleanPagePdfICTRtranscript(elem) for elem in text if elem != ""]
    print(f"Num of sentences with cleaning {len(text)}\n")    
    

Path input/transcripts/ictr/TRS15978R0000616228.DOC
Num of sentences 1883
Num of sentences with cleaning 1635



#### Get list of documents (either from excel file or default values)

In [4]:
if GLB_GET_LIST_FILES_ICTR_FROM_EXCEL_FILE:
    PATH_ICTR_EXCEL_LIST_FILE_ICTR_TRANSCRIPTS = "input/html-links-ICTR.xlsx"

    #df = pd.read_excel(PATH_ICTR_EXCEL_LIST_FILE_ICTR_TRANSCRIPTS, index_col=None, names=["url", "court", "case", "date", "person_name"]) #v1 of file
    df = pd.read_excel(PATH_ICTR_EXCEL_LIST_FILE_ICTR_TRANSCRIPTS, index_col=None, names=["url", "court", "accused", "case", "date", "witness"])    

    list_url_cases_ictr = list(df["url"])
else:
    list_url_cases_ictr = [transcript_eccc_case_url]
    
print(f"Length of elements in list_url_cases_ictr is {len(list_url_cases_ictr)}")

Length of elements in list_url_cases_ictr is 38


#### Save documents

In [8]:
counter = 0
pattern_was_found = False

for index_case, url_html_case in enumerate(list_url_cases_ictr):
    list_all_sentences = list()
    
    response = requests.get(url_html_case)
    doc_name = url_html_case[url_html_case.rindex("/")+1:]
    
    f = open(join(GLB_PATH_INPUT_DIRECTORY_ICTR, doc_name), GLB_FILE_BINARY_OP)
    f.write(response.content)
    f.close()
    
    text = textract.process(join(GLB_PATH_INPUT_DIRECTORY_ICTR, doc_name))
    text = text.decode("utf-8")
    counter_list_all_sentences = len(text.split('\n'))
    
    # Consider from PROCEEDINGS
    patternWasFound = False
    for PATTERN_PROCEEDINGS in cleaning_transc.GLB_ECCC_PATTERN_BEGIN_CONTENT_LIST:
        if not patternWasFound:
            index_pattern = text.index(PATTERN_PROCEEDINGS) if PATTERN_PROCEEDINGS in text else -1
            if index_pattern >= 0:
                patternWasFound = True
                text = text[index_pattern:]
                break
    
    text = text.split("\n")
    text = [cleaning_transc.cleanPagePdfICTRtranscript(elem) for elem in text if elem != ""]
    
    id_case = url_html_case[url_html_case.rindex("/")+1: url_html_case.rindex(".")]
    f = open(join(GLB_PATH_OUTPUT_DIRECTORY_ICTR, GLB_COURT_PREFIX_FILE_ICTR + id_case + GLB_EXTENSION_TXT), GLB_FILE_WRITE_OP, encoding=GLB_FILE_ENCODING_UTF8)
    
    counter = 0
    for index, sentence in enumerate(text):
        clean_paragraph = cleaning_transc.cleanSentenceECCCtranscript(sentence)
        if clean_paragraph != cleaning_transc.GLB_EMPTY_STRING:
            counter+=1
            f.write(clean_paragraph + GLB_CHAR_NEWLINE)
    f.close()
    
    
    print(f'{index_case+1}) Number of retrieved paragraphs of transcript {url_html_case} is {counter_list_all_sentences} was reduced to {counter}')

1) Number of retrieved paragraphs of transcript https://ucr.irmct.org/LegalRef/CMSDocStore/Public/English/Transcript/NotIndexable/ICTR-98-44/TRS15978R0000616228.DOC is 1883 was reduced to 1635
2) Number of retrieved paragraphs of transcript https://ucr.irmct.org/LegalRef/CMSDocStore/Public/English/Transcript/NotIndexable/ICTR-98-44/TRS17266R0000616529.DOC is 2619 was reduced to 2285
3) Number of retrieved paragraphs of transcript https://ucr.irmct.org/LegalRef/CMSDocStore/Public/English/Transcript/NotIndexable/ICTR-98-44/TRS19783R0000617360.DOC is 864 was reduced to 656
4) Number of retrieved paragraphs of transcript https://ucr.irmct.org/LegalRef/CMSDocStore/Public/English/Transcript/NotIndexable/ICTR-98-44/TRS17087R0000617365.DOC is 3595 was reduced to 3178
5) Number of retrieved paragraphs of transcript https://ucr.irmct.org/LegalRef/CMSDocStore/Public/English/Transcript/NotIndexable/ICTR-98-44/TRS18066R0000617403.DOC is 3747 was reduced to 3171
6) Number of retrieved paragraphs of 